## Step 0: Importing Libraries and Installation

In [1]:
!pip install sentence-transformers chromadb

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 3.1 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.4/21.4 MB 120.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 278.2/278.2 kB 19.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 52.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 103.3/103.3 kB 8.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.4/17.4 MB 126.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.5/72.5 kB 5.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.3/132.3 kB 9.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.9/65.9 kB 4.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 208.0/208.0 kB 15.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 105.4/105.4 kB 8.4 MB/s et

In [ ]:
import pandas as pd
import json
import chromadb
from chromadb.config import Settings

## Step 1: Load Dataset

In [5]:
incident_cases = pd.read_json("incident_cases_preprocessed.json")
incident_cases.head()

,id,title,road_name,area,segment_id,time_of_day,weather,cause,severity_label,vehicles_involved,...,response_taken,resolution_time_minutes,secondary_incident,description_cleaned,entities,tokens,tokens_clean,tokens_lemmatized,tokens_ngram,text_processed
0,1,High severity accident on King Road,King Road,Intersection 1,S001,morning_peak,rainy,rear_end_collision,high,3,...,dispatch_ambulance,60,False,high severity rearend collision on king road n...,"[[1, CARDINAL], [morning, TIME]]","[high, severity, rearend, collision, on, king,...","[high, severity, rearend, collision, king, roa...","[high, severity, rearend, collision, king, roa...","[high_severity, rearend, collision, king, road...",high severity rearend collision king road near...
1,2,Medium congestion on Tahlia Street,Tahlia Street,Intersection 2,S002,evening_peak,clear,event_crowd,medium,0,...,reroute_traffic,45,False,medium congestion on tahlia street near inters...,"[[2, CARDINAL], [evening, TIME]]","[medium, congestion, on, tahlia, street, near,...","[medium, congestion, tahlia, street, near, int...","[medium, congestion, tahlia, street, near, int...","[medium, congestion, tahlia, street, near_inte...",medium congestion tahlia street near intersect...
2,3,Critical ambulance delay on Haram Road,Haram Road,Intersection 4,S004,morning_peak,foggy,congestion,critical,1,...,manual_signal_override,25,False,critical delay of an ambulance on haram road n...,"[[4, CARDINAL]]","[critical, delay, of, an, ambulance, on, haram...","[critical, delay, ambulance, haram, road, near...","[critical, delay, ambulance, haram, road, near...","[critical, delay, ambulance, haram, road_near,...",critical delay ambulance haram road near inter...
3,4,Low severity breakdown on Airport Road,Airport Road,Intersection 5,S005,midday,clear,breakdown,low,1,...,dispatch_tow_truck,30,False,single vehicle breakdown on the shoulder of ai...,"[[5, CARDINAL], [midday, TIME]]","[single, vehicle, breakdown, on, the, shoulder...","[single, vehicle, breakdown, shoulder, airport...","[single, vehicle, breakdown, shoulder, airport...","[single, vehicle, breakdown, shoulder, airport...",single vehicle breakdown shoulder airport road...
4,5,High severity multi-vehicle crash on Industria...,Industrial Road,Intersection 12,S012,morning_peak,dusty,multi_vehicle_collision,high,4,...,dispatch_ambulance,90,True,high severity multivehicle crash on industrial...,"[[high severity multivehicle, ORG], [12, CARDI...","[high, severity, multivehicle, crash, on, indu...","[high, severity, multivehicle, crash, industri...","[high, severity, multivehicle, crash, industri...","[high_severity, multivehicle, crash, industria...",high severity multivehicle crash industrial ro...


In [6]:
road_rules = pd.read_json("road_rules_preprocessed.json")
road_rules.head()

,id,road_name,segment_id,area,speed_limit_kmh,min_speed_kmh,lanes,road_type,allowed_vehicles,has_emergency_lane,typical_congestion_level,peak_period,notes,notes_cleaned,tokens,tokens_clean,tokens_lemmatized,text_processed
0,1,King Road,S001,Intersection 1,80,40,3,arterial,"[car, truck, bus, motorcycle, emergency]",True,high,morning_peak,"Segment of King Road near Intersection 1, ofte...",segment of king road near intersection 1 often...,"[segment, of, king, road, near, intersection, ...","[segment, king, road, near, intersection, 1, o...","[segment, king, road, near, intersection, 1, o...",segment king road near intersection 1 often co...
1,2,Tahlia Street,S002,Intersection 2,60,30,2,collector,"[car, bus, motorcycle, emergency]",False,medium,evening_peak,Commercial corridor with shops and restaurants...,commercial corridor with shops and restaurants...,"[commercial, corridor, with, shops, and, resta...","[commercial, corridor, shops, restaurants, eve...","[commercial, corridor, shop, restaurant, eveni...",commercial corridor shop restaurant evening co...
2,3,Prince Sultan Road,S003,Intersection 3,70,40,3,arterial,"[car, truck, bus, motorcycle, emergency]",True,medium,morning_peak,Major north–south corridor connecting resident...,major northsouth corridor connecting residenti...,"[major, northsouth, corridor, connecting, resi...","[major, northsouth, corridor, connecting, resi...","[major, northsouth, corridor, connecting, resi...",major northsouth corridor connecting residenti...
3,4,Haram Road,S004,Intersection 4,80,50,4,highway,"[car, truck, bus, emergency]",True,high,all_day,High-demand corridor serving pilgrims and loca...,highdemand corridor serving pilgrims and local...,"[highdemand, corridor, serving, pilgrims, and,...","[highdemand, corridor, serving, pilgrims, loca...","[highdemand, corridor, serving, pilgrim, local...",highdemand corridor serving pilgrim local resi...
4,5,Airport Road,S005,Intersection 5,100,60,4,highway,"[car, truck, bus, emergency]",True,medium,morning_peak,"Main access route to the airport, frequent eme...",main access route to the airport frequent emer...,"[main, access, route, to, the, airport, freque...","[main, access, route, airport, frequent, emerg...","[main, access, route, airport, frequent, emerg...",main access route airport frequent emergency v...


In [7]:
traffic_policies = pd.read_json("traffic_policies_preprocessed.json")
traffic_policies.head()

,id,policy_code,category,applicable_severity,description,priority_rank,related_incident_types,time_window,applies_to_roads,description_cleaned,tokens,tokens_clean,tokens_lemmatized,text_processed
0,1,EMERGENCY_PRIORITY_HIGH,emergency_priority,high,When an emergency vehicle is detected within 2...,1,"[emergency_delay, accident]",all_day,"[Haram Road, Hospital Street, Airport Road]",when an emergency vehicle is detected within 2...,"[when, an, emergency, vehicle, is, detected, w...","[emergency, vehicle, detected, within, 200, me...","[emergency, vehicle, detected, within, 200, me...",emergency vehicle detected within 200 meter in...
1,2,EMERGENCY_PRIORITY_MEDIUM,emergency_priority,medium,When an emergency vehicle is approaching with ...,2,[emergency_delay],morning_peak,"[Ring Road East, Ring Road West]",when an emergency vehicle is approaching with ...,"[when, an, emergency, vehicle, is, approaching...","[emergency, vehicle, approaching, moderate, co...","[emergency, vehicle, approaching, moderate, co...",emergency vehicle approaching moderate congest...
2,3,EMERGENCY_PRIORITY_LOW,emergency_priority,low,"Under low congestion, give normal green but av...",3,[emergency_delay],night,"[King Road, Corniche Road]",under low congestion give normal green but avo...,"[under, low, congestion, give, normal, green, ...","[low, congestion, give, normal, green, avoid, ...","[low, congestion, give, normal, green, avoid, ...",low congestion give normal green avoid unneces...
3,4,CONGESTION_MITIGATION_HIGH,congestion_mitigation,high,If average speed on a segment drops below 10 k...,1,[congestion],evening_peak,"[King Road, Market Road, Industrial Road]",if average speed on a segment drops below 10 k...,"[if, average, speed, on, a, segment, drops, be...","[average, speed, segment, drops, 10, kmh, dens...","[average, speed, segment, drop, 10, kmh, densi...",average speed segment drop 10 kmh density high...
4,5,CONGESTION_MITIGATION_MEDIUM,congestion_mitigation,medium,If average speed is between 10 and 25 km/h wit...,2,"[congestion, event_crowd]",all_day,"[Tahlia Street, Corniche Road]",if average speed is between 10 and 25 kmh with...,"[if, average, speed, is, between, 10, and, 25,...","[average, speed, 10, 25, kmh, medium, queues, ...","[average, speed, 10, 25, kmh, medium, queue, s...",average speed 10 25 kmh medium queue slightly ...


In [8]:
emergency_protocols = pd.read_json("emergency_protocols_preprocessed.json")
emergency_protocols.head()

,id,protocol_code,emergency_type,default_severity,description,steps,notify_services,max_response_time_minutes,requires_manual_confirmation,description_cleaned,tokens,tokens_clean,tokens_lemmatized,text_processed
0,1,MAJOR_ACCIDENT,major_accident,critical,Collision involving multiple vehicles with pos...,[Verify exact location using GPS and nearest i...,"[police, ambulance, najm]",10,False,collision involving multiple vehicles with pos...,"[collision, involving, multiple, vehicles, wit...","[collision, involving, multiple, vehicles, pos...","[collision, involving, multiple, vehicle, poss...",collision involving multiple vehicle possible ...
1,2,DELAYED_EMERGENCY_VEHICLE,delayed_emergency_vehicle,high,"An ambulance, fire truck, or police vehicle is...",[Identify blocked segments on the planned emer...,"[ambulance, police, traffic_control_center]",5,False,an ambulance fire truck or police vehicle is s...,"[an, ambulance, fire, truck, or, police, vehic...","[ambulance, fire, truck, police, vehicle, sign...","[ambulance, fire, truck, police, vehicle, sign...",ambulance fire truck police vehicle significan...
2,3,FIRE_OR_SMOKE_NEAR_ROAD,fire_or_smoke,high,Presence of smoke or fire near the roadway det...,[Confirm smoke or fire levels using camera fee...,"[fire_department, ambulance, police]",8,True,presence of smoke or fire near the roadway det...,"[presence, of, smoke, or, fire, near, the, roa...","[presence, smoke, fire, near, roadway, detecte...","[presence, smoke, fire, near, roadway, detecte...",presence smoke fire near roadway detected sens...
3,4,FLOODING_ROAD,flooding_road,medium,"Road segment partially or fully flooded, posin...",[Mark flooded road segments as closed in the k...,"[municipality, police]",20,True,road segment partially or fully flooded posing...,"[road, segment, partially, or, fully, flooded,...","[road, segment, partially, fully, flooded, pos...","[road, segment, partially, fully, flooded, pos...",road segment partially fully flooded posing ri...
4,5,MINOR_ACCIDENT,minor_accident,medium,Two-vehicle collision with no serious injuries...,[Verify that there are no serious injuries usi...,"[police, najm]",15,False,twovehicle collision with no serious injuries ...,"[twovehicle, collision, with, no, serious, inj...","[twovehicle, collision, serious, injuries, pot...","[twovehicle, collision, serious, injury, poten...",twovehicle collision serious injury potential ...




---



## Step 2: RAG content

We’ll use `text_processed` to choose what to index


In [9]:
def safe_get_text(df, processed_col, fallback_cols):
    texts = []
    for _, row in df.iterrows():
        if processed_col in df.columns and isinstance(row[processed_col], str) and row[processed_col].strip():
            texts.append(row[processed_col])
        else:
            # fallback: join some original columns
            parts = []
            for col in fallback_cols:
                if col in df.columns and isinstance(row[col], str):
                    parts.append(row[col])
            texts.append(" ".join(parts))
    return texts

# Build text lists
incident_texts = safe_get_text(
    incident_cases,
    "text_processed",
    ["description", "title"]
)

road_rules_texts = safe_get_text(
    road_rules,
    "text_processed",
    ["notes", "road_name", "area"]
)

policy_texts = safe_get_text(
    traffic_policies,
    "text_processed",
    ["description", "policy_code"]
)

protocol_texts = safe_get_text(
    emergency_protocols,
    "text_processed",
    ["description", "protocol_code"]
)

len(incident_texts), len(road_rules_texts), len(policy_texts), len(protocol_texts)


(12, 15, 12, 5)

## 2.1: Prepare one combined corpus for RAG

`all_texts` will create a list of document strings <br>
`all_metadata` will create a list of metadata dicts (source, id, etc.) <br>
`all_ids` will create unique ids (strings) <br>

In [10]:
all_texts = []
all_metadata = []
all_ids = []

# 1) Incidents
for i, text in enumerate(incident_texts):
    all_texts.append(text)
    all_metadata.append({
        "source": "incident_case",
        "row_id": int(incident_cases.iloc[i]["id"]),
        "severity_label": str(incident_cases.iloc[i]["severity_label"]),
        "road_name": str(incident_cases.iloc[i]["road_name"])
    })
    all_ids.append(f"incident_{incident_cases.iloc[i]['id']}")

# 2) Road rules
for i, text in enumerate(road_rules_texts):
    all_texts.append(text)
    all_metadata.append({
        "source": "road_rule",
        "row_id": int(road_rules.iloc[i]["id"]),
        "road_name": str(road_rules.iloc[i]["road_name"]),
        "segment_id": str(road_rules.iloc[i]["segment_id"])
    })
    all_ids.append(f"road_rule_{road_rules.iloc[i]['id']}")

# 3) Traffic policies
for i, text in enumerate(policy_texts):
    all_texts.append(text)
    all_metadata.append({
        "source": "traffic_policy",
        "row_id": int(traffic_policies.iloc[i]["id"]),
        "policy_code": str(traffic_policies.iloc[i]["policy_code"]),
        "category": str(traffic_policies.iloc[i]["category"])
    })
    all_ids.append(f"policy_{traffic_policies.iloc[i]['id']}")

# 4) Emergency protocols
for i, text in enumerate(protocol_texts):
    all_texts.append(text)
    all_metadata.append({
        "source": "emergency_protocol",
        "row_id": int(emergency_protocols.iloc[i]["id"]),
        "protocol_code": str(emergency_protocols.iloc[i]["protocol_code"]),
        "emergency_type": str(emergency_protocols.iloc[i]["emergency_type"])
    })
    all_ids.append(f"protocol_{emergency_protocols.iloc[i]['id']}")

len(all_texts), len(all_metadata), len(all_ids)


(44, 44, 44)

## 2.2: Create embeddings using `sentence-transformers`

In [ ]:
from sentence_transformers import SentenceTransformer

embed_model = SentenceTransformer("all-MiniLM-L6-v2") # or "all-MiniLM-L12-v2", but using L6 for speed for small datasets

embeddings = embed_model.encode(all_texts, show_progress_bar=True)
len(embeddings), len(embeddings[0])

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

Batches:   0%|          | 0/2 [00:00<?, ?it/s]

(44, 384)

## 2.3: Create a ChromaDB persistent index

Here we build the actual RAG index.

In [ ]:
# Create a persistent client (will create a folder "rag_index")
client = chromadb.PersistentClient(path="rag_index")

collection = client.get_or_create_collection(
    name="traffic_knowledge"
)

# Clear collection if you re-run
if collection.count() > 0:
    # NOTE: only if you want to reset
    ids_to_delete = collection.get()["ids"]
    collection.delete(ids=ids_to_delete)

# Add documents to Chroma
collection.add(
    ids=all_ids,
    documents=all_texts,
    metadatas=all_metadata,
    embeddings=embeddings.tolist()  # Chroma expects normal Python lists
)

print("Documents in collection:", collection.count())


Documents in collection: 44


## 2.4: Testing queries

In [ ]:
def embed_query(text: str):
    return embed_model.encode([text])[0].tolist()

query = "What is the speed limit on King Road?"
query_embedding = embed_query(query)

results = collection.query(
    query_embeddings=[query_embedding],
    n_results=3
)

results

{'ids': [['incident_1', 'road_rule_1', 'incident_7']],
 'embeddings': None,
 'documents': [['high severity rearend collision king road near intersection 1 morning peak rainy condition multiple lane partially blocked shockwave congestion spreading upstream',
   'segment king road near intersection 1 often congested morning peak due commuter',
   'low severity slowdown corniche road near intersection 6 due recreational evening traffic incident blockage']],
 'uris': None,
 'included': ['metadatas', 'documents', 'distances'],
 'data': None,
 'metadatas': [[{'row_id': 1,
    'severity_label': 'high',
    'source': 'incident_case',
    'road_name': 'King Road'},
   {'source': 'road_rule',
    'segment_id': 'S001',
    'road_name': 'King Road',
    'row_id': 1},
   {'severity_label': 'low',
    'source': 'incident_case',
    'row_id': 7,
    'road_name': 'Corniche Road'}]],
 'distances': [[0.9949946403503418, 1.0206968784332275, 1.0524752140045166]]}

In [ ]:
for doc, meta in zip(results["documents"][0], results["metadatas"][0]): # for better formatting
    print("SOURCE:", meta["source"])
    print("META:", meta)
    print("TEXT:", doc)
    print("-" * 80)


SOURCE: incident_case
META: {'row_id': 1, 'severity_label': 'high', 'source': 'incident_case', 'road_name': 'King Road'}
TEXT: high severity rearend collision king road near intersection 1 morning peak rainy condition multiple lane partially blocked shockwave congestion spreading upstream
--------------------------------------------------------------------------------
SOURCE: road_rule
META: {'source': 'road_rule', 'segment_id': 'S001', 'road_name': 'King Road', 'row_id': 1}
TEXT: segment king road near intersection 1 often congested morning peak due commuter
--------------------------------------------------------------------------------
SOURCE: incident_case
META: {'severity_label': 'low', 'source': 'incident_case', 'row_id': 7, 'road_name': 'Corniche Road'}
TEXT: low severity slowdown corniche road near intersection 6 due recreational evening traffic incident blockage
--------------------------------------------------------------------------------
